In [2]:
import pandas as pd
import numpy as np
import sktime
import seaborn as snsc
import matplotlib.pyplot as plt
from convertcsv.import_preprocess_v4 import readcsvs
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sktime.transformations.panel.rocket import Rocket
from sktime.transformations.panel.rocket import MiniRocketMultivariate
from sktime.classification.kernel_based import RocketClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.impute import KNNImputer
from convertcsv.get_all_metrics_with_tags import get_all_metrics_with_tags
from visualization import graphs
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sktime.datatypes import convert_to
from sktime.datatypes import MTYPE_REGISTER
from collections import Counter


In [3]:
#What should we do here?

#Set up many different pipelines to compare.

#The things to vary, in order of significance/importance
"""
NaN treshold
"""
file_list, y = get_all_metrics_with_tags(r"F:\Master\Kubernetes\sockshop\microservices-demo\query\automated\generated_csvs_4")
initial_readings = readcsvs(file_list, reduce_NaNs_treshold=False, remove_unique_cols=True, remove_monotonic_increasing=True)
initial_readings



go_gc_duration_seconds&payment:80&payment&0.25  \
instances timepoints                                                            
0         2023-03-24 14:09:48                                        0.000307   
          2023-03-24 14:09:53                                        0.000307   
          2023-03-24 14:09:58                                        0.000307   
          2023-03-24 14:10:03                                        0.000307   
          2023-03-24 14:10:08                                        0.000307   
...                                                                       ...   
75        2023-03-29 16:21:34                                        0.000318   
          2023-03-29 16:21:39                                        0.000318   
          2023-03-29 16:21:44                                        0.000318   
          2023-03-29 16:21:49                                        0.000318   
          2023-03-29 16:21:54                                        0.000318   

                               go_gc_duration_seconds&payment:80&payment&0.75  \
instances timepoints                                                            
0         2023-03-24 14:09:48                                        0.000333   
          2023-03-24 14:09:53                                        0.000333   
          2023-03-24 14:09:58                                        0.000333   
          2023-03-24 14:10:03                                        0.000333   
          2023-03-24 14:10:08                                        0.000333   
...                                                                       ...   
75        2023-03-29 16:21:34                                        0.000370   
          2023-03-29 16:21:39                                        0.000370   
          2023-03-29 16:21:44                                        0.000370   
          2023-03-29 16:21:49                                        0.000370   
          2023-03-29 16:21:54                                        0.000370   

                               go_gc_duration_seconds&user:80&user&0.25  \
instances timepoints                                                      
0         2023-03-24 14:09:48                                  0.000321   
          2023-03-24 14:09:53                                  0.000321   
          2023-03-24 14:09:58                                  0.000321   
          2023-03-24 14:10:03                                  0.000321   
          2023-03-24 14:10:08                                  0.000321   
...                                                                 ...   
75        2023-03-29 16:21:34                                  0.000380   
          2023-03-29 16:21:39                                  0.000380   
          2023-03-29 16:21:44                                  0.000380   
          2023-03-29 16:21:49                                  0.000380   
          2023-03-29 16:21:54                                  0.000380   

                               go_gc_duration_seconds&user:80&user&0.5  \
instances timepoints                                                     
0         2023-03-24 14:09:48                                 0.000333   
          2023-03-24 14:09:53                                 0.000333   
          2023-03-24 14:09:58                                 0.000333   
          2023-03-24 14:10:03                                 0.000333   
          2023-03-24 14:10:08                                 0.000333   
...                                                                ...   
75        2023-03-29 16:21:34                                 0.000451   
          2023-03-29 16:21:39                                 0.000451   
          2023-03-29 16:21:44                                 0.000451   
          2023-03-29 16:21:49                                 0.000451   
          2023-03-29 16:21:54                              

In [4]:
#Prior testing revealed optimal KNN imputation
imputer = KNNImputer(n_neighbors=1)
complete_value_set = imputer.fit_transform(initial_readings)
imputed_df = pd.DataFrame(complete_value_set, index=initial_readings.index, columns=initial_readings.columns)
imputed_df.index.get_level_values(1)

DatetimeIndex(['2023-03-24 14:09:48', '2023-03-24 14:09:53',
               '2023-03-24 14:09:58', '2023-03-24 14:10:03',
               '2023-03-24 14:10:08', '2023-03-24 14:10:13',
               '2023-03-24 14:10:18', '2023-03-24 14:10:23',
               '2023-03-24 14:10:28', '2023-03-24 14:10:33',
               ...
               '2023-03-29 16:21:09', '2023-03-29 16:21:14',
               '2023-03-29 16:21:19', '2023-03-29 16:21:24',
               '2023-03-29 16:21:29', '2023-03-29 16:21:34',
               '2023-03-29 16:21:39', '2023-03-29 16:21:44',
               '2023-03-29 16:21:49', '2023-03-29 16:21:54'],
              dtype='datetime64[ns]', name='timepoints', length=10073, freq=None)

In [5]:
#split into training x and y
X_train, X_test, y_train, y_test = train_test_split(convert_to(imputed_df,to_type="df-list"), y)
print(MTYPE_REGISTER)
#X_train = convert_to(X_train, to_type="pd-multiindex")


[('pd.Series', 'Series', 'pd.Series representation of a univariate series'), ('pd.DataFrame', 'Series', 'pd.DataFrame representation of a uni- or multivariate series'), ('np.ndarray', 'Series', '2D numpy.ndarray with rows=samples, cols=variables, index=integers'), ('xr.DataArray', 'Series', 'xr.DataArray representation of a uni- or multivariate series'), ('dask_series', 'Series', 'xdas representation of a uni- or multivariate series'), ('nested_univ', 'Panel', 'pd.DataFrame with one column per variable, pd.Series in cells'), ('numpy3D', 'Panel', '3D np.array of format (n_instances, n_columns, n_timepoints)'), ('numpyflat', 'Panel', 'WARNING: only for internal use, not a fully supported Panel mtype. 2D np.array of format (n_instances, n_columns*n_timepoints)'), ('pd-multiindex', 'Panel', 'pd.DataFrame with multi-index (instances, timepoints)'), ('pd-wide', 'Panel', 'pd.DataFrame in wide format, cols = (instance*timepoints)'), ('pd-long', 'Panel', 'pd.DataFrame in long format, cols = (in

Padding/reduction

In [40]:
#Find the length of each individual time series
def trimming(df:pd.DataFrame, y, min_percent=90):
    instances = df.index.get_level_values(0).unique()
    timeSeriesLengths = []
    for instance in instances:
        #This line of code gets every second level index value that corresponds to the current first level index value. 
        second_level_values = df.index.get_level_values(1)[df.index.get_level_values(0)==instance]
        length = len(second_level_values)
        timeSeriesLengths.append(length)
    #Find the most common value.
    counts = Counter(timeSeriesLengths)
    most_common_value, most_common_value_count = counts.most_common(1)[0]
    percentage = (most_common_value_count / len(timeSeriesLengths)) *100

    if percentage >= min_percent:
        indicies_to_drop = []
        for indice in range(len(timeSeriesLengths)):
            if timeSeriesLengths[indice] != most_common_value:
                indicies_to_drop.append(indice)
        print()
        filtered_df:pd.DataFrame = df[~df.index.get_level_values(0).isin(indicies_to_drop)]
        print(filtered_df.index[0])
        #Reset the index
        current_indice = 0
        desired_indice = 0
        newindex_tuples = []
        for index_tuple in filtered_df.index:
            #The thing to watch out for is the fact that we have to keep track of both the 
            #desired index and the index being displayed in the tuple. 
            #When the tuple being looked at changes, the index increases.
            #If two indexes have been removed, the issue becomes keeping track of that. 
            #do a while loop to update it immediately. 
            #nevermind just set it equals, we only care about change anyway
            if(index_tuple[0] != current_indice):
                #this way, when the index being looked at changes, we know.
                current_indice = index_tuple[0]
                #we logged that there is a change so we know its time for next indice
                desired_indice += 1
 
            newindex_tuples.append((desired_indice, index_tuple[1]))
        
        newindex = pd.MultiIndex.from_tuples(newindex_tuples, names=filtered_df.index.names)
        newdf = pd.DataFrame(filtered_df.values, columns=filtered_df.columns, index=newindex)
            
        y = np.array(y)

        #Now rebuild the multiindex to be omontonically increasing.
        
        
        #This still keeps the fucky index in the thing

        return newdf, np.delete(y, indicies_to_drop)

    return ValueError(percentage)
    #To properly preprocess, one should check if the most common value equals the max value. If it does, great.
    #Imagine we find a perfect thing for it. Now we have a couple of datasets consisting of multivariate data.
    #It's important to preserve the general shape of the dataset. So just adding mean to the front and back probably isn't that good. 
    #Can decide to simply purge the ones that aren't of the correct length. To do this, you would have to decide that the most common value is the highest one, and that there are very few time series that differ. Incurs data loss but saves a lot of time.
    #Instead of getting bogged down in a custom optimal solution, get the alright one. For now, get a determiner that purging the ones that are too small is ok. 
    #If ok, proceed. Can then later compare with other algorithms to see if the purging was helpful.
    #Write in the thesis that shit happens and time constraints wcyd

    #For now, determine if purging is ait
    #The most common value should be at least 90% to only tolerate 10% data loss perhaps.

trimmed_df , trimmed_y = trimming(imputed_df, y)
trimmed_df


(0, Timestamp('2023-03-24 14:09:48'))


go_gc_duration_seconds&payment:80&payment&0.25  \
instances timepoints                                                            
0         2023-03-24 14:09:48                                        0.000307   
          2023-03-24 14:09:53                                        0.000307   
          2023-03-24 14:09:58                                        0.000307   
          2023-03-24 14:10:03                                        0.000307   
          2023-03-24 14:10:08                                        0.000307   
...                                                                       ...   
70        2023-03-29 16:21:34                                        0.000318   
          2023-03-29 16:21:39                                        0.000318   
          2023-03-29 16:21:44                                        0.000318   
          2023-03-29 16:21:49                                        0.000318   
          2023-03-29 16:21:54                                        0.000318   

                               go_gc_duration_seconds&payment:80&payment&0.75  \
instances timepoints                                                            
0         2023-03-24 14:09:48                                        0.000333   
          2023-03-24 14:09:53                                        0.000333   
          2023-03-24 14:09:58                                        0.000333   
          2023-03-24 14:10:03                                        0.000333   
          2023-03-24 14:10:08                                        0.000333   
...                                                                       ...   
70        2023-03-29 16:21:34                                        0.000370   
          2023-03-29 16:21:39                                        0.000370   
          2023-03-29 16:21:44                                        0.000370   
          2023-03-29 16:21:49                                        0.000370   
          2023-03-29 16:21:54                                        0.000370   

                               go_gc_duration_seconds&user:80&user&0.25  \
instances timepoints                                                      
0         2023-03-24 14:09:48                                  0.000321   
          2023-03-24 14:09:53                                  0.000321   
          2023-03-24 14:09:58                                  0.000321   
          2023-03-24 14:10:03                                  0.000321   
          2023-03-24 14:10:08                                  0.000321   
...                                                                 ...   
70        2023-03-29 16:21:34                                  0.000380   
          2023-03-29 16:21:39                                  0.000380   
          2023-03-29 16:21:44                                  0.000380   
          2023-03-29 16:21:49                                  0.000380   
          2023-03-29 16:21:54                                  0.000380   

                               go_gc_duration_seconds&user:80&user&0.5  \
instances timepoints                                                     
0         2023-03-24 14:09:48                                 0.000333   
          2023-03-24 14:09:53                                 0.000333   
          2023-03-24 14:09:58                                 0.000333   
          2023-03-24 14:10:03                                 0.000333   
          2023-03-24 14:10:08                                 0.000333   
...                                                                ...   
70        2023-03-29 16:21:34                                 0.000451   
          2023-03-29 16:21:39                                 0.000451   
          2023-03-29 16:21:44                                 0.000451   
          2023-03-29 16:21:49                                 0.000451   
          2023-03-29 16:21:54                              

In [8]:
converted = convert_to(trimmed_df,to_type="df-list")
X_train, X_test, y_train, y_test = train_test_split(convert_to(trimmed_df,to_type="df-list"),y)

KeyError: 4

In [ ]:
rocketclass = RocketClassifier()

rocketclass.fit(X=X_train, y=np.array(y_train))
rocketclass.score(X_test, np.array(y_test))


ValueError: Data seen by RocketClassifier instance has unequal length series, but this RocketClassifier instance cannot handle unequal length series. Calls with unequal length series may result in error or unreliable results.